In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
import os
from bib import *

### Load JACUSA CALL2 features

In [19]:
# Params from snakemake
inp1 = snakemake.input[0]       # WT vs IVT JACUSA CALL2 Output
inp2 = snakemake.input[1]       # KO vs IVT JACUSA CALL2 Output
lof_thre = snakemake.params[0]  # LOF contamination value
lof_neigh = snakemake.params[1] # LOF neighborhood size
method = snakemake.params[2]    # NR_003286_RNA18SN5 or NR_003287_RNA28SN5
mod_status_file = snakemake.params[3] # rRNA modification status file

output =  snakemake.output[0] # output folder

dic = dict({'control_cond1':inp1 , 'control_cond2' : inp2})

if not os.path.exists(output):
    os.makedirs(output, exist_ok=True)
output = output + "/"

In [10]:
# JACUSA CALL2 features
for key in dic:
    
    df0 = pd.read_csv(dic[key], sep = '\t',skiprows=1)
    df0 = ExtractFeatures(df0)
#     df0.to_csv(JC2out_Features, index=False) 
    df0['Ref_Pos'] = df0["Ref"]+ "_" + df0["Pos"].astype(str) 

    # load rRNA modifications
#     mod = pd.read_csv(mod_file, sep = '\t', header = None)
#     mod['Ref_Pos'] = mod[0]+ "_" + mod[2].astype(str) 
#     mod = mod.rename(columns={3 :  'Mod'})

    # load rRNA modifications status
    mods = pd.read_csv(mod_status_file, sep = ',')
    mods['Ref_Pos'] = mods[mods.columns[0]]+ "_" + mods[mods.columns[1]].astype(str) 
#     mod_ = pd.merge(mods[['Ref_Pos','ModStatus','Status']],mod[['Ref_Pos','Mod']], on='Ref_Pos')
    # merge features with modifications
    df1 = pd.merge(df0,mods, on='Ref_Pos')
    df1 = df1.sort_values(by=['Ref' , 'Pos']).reset_index(drop=True)
    
    ### Add features in 5mer context
    """ 
    Build the table of features in 5mer context : Mismatch, Mismatch + Insertion + Deletion, 
    Mismatch in the 5mer context + Insertion + Deletion, Mismatch + Insertion + Deletion all in the 5mer context   
    """

    df2 = KmerFeatures(df1)
    feat1 = df2.Mis
    feat2 = df2.Mis + df2.Ins + df2.Del
    feat3 = df2.SumMis + df2.Ins + df2.Del
    feat4 = df2.SumMis + df2.SumIns +df2.SumDel

    dfsave = pd.DataFrame({'label':key,'Ref_Pos': df2['Ref_Pos'],'Ref': df2['Ref'], 'Pos':df2['Pos'],'Coverage' : df2['Min_cov'],'Mis': feat1, 'Mis+Del+Ins': feat2
                           , 'MisContext+Del+Ins':feat3, 'Mis+Del+Ins_Context':feat4, 'ModStatus' : df2['ModStatus'], 'Status':df2['Status'] })

    # add features to the table of features
    if 'table' in globals():
        table = table.append(dfsave)
    else: 
        table = dfsave

In [27]:
table.to_csv(output+'Features_JACUSA2CALL2.csv', index=False)

Generate BarPlots for each features combination. Features are supposed to be already generated and added to the table of features

In [ ]:
features = ['Mis', 'Mis+Del+Ins', 'MisContext+Del+Ins','Mis+Del+Ins_Context']
REF =['NR_003287_RNA28SN5','NR_003286_RNA18SN5']
for ref in REF:
    for feature in features:
        label1 = table['label'].unique()[0]
        label2 = table['label'].unique()[1]

        x= table[(table["label"] == label1) & (table["Ref"] == ref)].reset_index(drop=True)
        y= table[(table["label"] == label2) & (table["Ref"] == ref)].reset_index(drop=True)

        BarPlot(x[['Pos',feature, 'ModStatus','Status']],feature,label1+' ('+ref+')',outlier=lof_thre,neigh=lof_neigh,path = output)
        BarPlot(y[['Pos',feature, 'ModStatus','Status']],feature,label2+' ('+ref+')',outlier=lof_thre,neigh=lof_neigh,path = output)

#      only outliers are labeled here ... ax.annotate() can be used to annotate other positions of interest. 